# BBC - Projet 2016
## Vaccin peptidique contre la bactérie Helicobacter Pylori

- Professeur: Carlos Peña (<a href="mailto:carlos.pena@heig-vd.ch">carlos.pena@heig-vd.ch</a>)
- Assistante: Aitana Lebrand (<a href="mailto:aitana.nevesdasilva@heig-vd.ch">aitana.nevesdasilva@heig-vd.ch</a>)

- Etudiants : Raphaël Racine et Nicolas Kobel

Date: 10 mai 2016
<br><br>

# Contexte

## Vaccin
Pour expliquer ce qu’est un vaccin, nous nous sommes inspirés de l’article suivant :
https://fr.wikipedia.org/wiki/Vaccination.

Un vaccin est une substance que l’on injecte dans un corps humain ou animal. Il permet de créer une réaction immunitaire positive contre une maladie infectieuse. 

C’est un antigène qui permet de stimuler le système immunitaire afin qu’il puisse s’activer de manière plus rapide et plus forte lors de la présence d’une vraie contamination.
Il existe plusieurs types de vaccins, mais on s’intéressera particulièrement au vaccin peptidique.

## Vaccin peptidique
Un vaccin peptidique permet d’injecter un ou plusieurs séquences d’acides aminés (courtes ou longues) d’antigènes, en y combinant des adjuvants.

### Adjuvant
Un adjuvant permet de renforcer ou stimuler un système immunitaire lorsqu’elle est administrée conjointement avec un antigène ( source : https://fr.wikipedia.org/wiki/Adjuvant ). Il s’agit d’un composant toxique.
Un antigène est caractérisé par son ensemble d’épitopes.

## Epitope
Pour expliquer ce qu’est un épitope, nous nous sommes inspirés de l’article suivant :
https://fr.wikipedia.org/wiki/%C3%89pitope.

Un épitope est un déterminant antigénique présentée sous forme de molécule reconnaissable par un paratope (partie variable d’un anticorps). 

Epitope est un synonyme de peptide dans le cas où celui-ci est reconnaissable par un paratope.

## Bactérie

Une bactérie est un micro-organisme capable de se reproduire. Il est à noter qu’il s’agit d’un organisme qui n’est ni végétal, ni animal.

Il s’agit d’un organisme procaryote, dont la structure cellulaire ne possède donc pas de noyau.

### Helicobacter Pilory
Pour expliquer ce qu’est la bactérie H Pylori, nous nous sommes inspirés de l’article suivant : http://www.e-sante.fr/helicobacter-pylori-bacterie-tous-dangers/actualite/1534.

La bactérie Helicobacter Pylori est une bactérie très petite responsable de plusieurs maladies de l’estomac, dont certaines peuvent aller jusqu’au cancer de l’estomac (qui commence en général par des ulcères).

Comme son nom l’indique, il s’agit d’une bactérie en forme d’hélice et sa particularité et de se sentir « chez elle » dans l’estomac malgré les niveaux d'acidité élevés dans cet environnement.	
Cette bactérie sécrète de l'uréase. Cette protéine permet à la bactérie de de transformer l'acide gastrique en ammoniaque anodin pour la bactérie. L'autre mécanisme de défense contre l'acide gastrique est la capacité de H. Pylori de pouvoir pénétrer la couche muqueuse de l'estomac.[1]

Cette couche muqueuse protège l'estomac, et donc le corps humain de sa propre acidité. Lorsque pénétrée par la bactérie, l'équilibre chimique de la muqueuse est dérangé. Ceci affaiblit la protection et peut mener à des ulcères voir des cancers.
Cette protéine est subdivisée en deux gènes appelés ureA et ureB. Ces deux gènes participent (avec d’autres gènes accessoires) à l’activité enzymatique de la bactérie.
[1]http://www.cancer.gov/about-cancer/causes-prevention/risk/infectious-agents/h-pylori-fact-sheet

## Protéine Uréase
La protéine Urease présente dans la bactérie H pilory a la forme suivante :
 
Source : http://www.rcsb.org/pdb/explore/explore.do?structureId=1e9y 

![](img/urease.png)

Elle forme un complexe supramoléculaire « Dodecameric » composé de 12 copies de couples (UreA, UreB) qui se trouve chacune dans un site actif, qui permet à la bactérie de survivre. Il y a donc 12 sites actifs dans cette protéine.

### Sous unité UreB
Source : http://www.rcsb.org/pdb/explore/explore.do?structureId=1e9y 

La sous-unité UreB est celle qui nous intéresse car c’est à partir de celle-ci qu’il sera possible de prédire des épitopes potentiellement utilisables dans le vaccin.

Cette sous-unité possède actuellement 5 mutations différentes : <b>K219X, L324K, I355A, I522V, D531N.</b>

Elle possède les gènes suivants : <b>ureB, hpuB, HP_0072</b>.

On peut trouver des informations (notamment sur les séquences de cette sous-unité UreB) à partir de la base de données UniprotKG (Swiss-Prot) sur le lien suivant : http://www.uniprot.org/uniprot/P69996

![](img/structure_ureb.png)

# Etat de l'art

## Diagnostique / Détection

En règle générale un dépistage de H. Pylori se fait lorsqu’un patient présente certains des symptômes suivants [1]:
<ul>
    <li>peines abdominales
	<li>nausées et vomissements
	<li>anémie
	<li>halitose
</ul>

### Outils de diagnostique
Les outils de diagnostiques les plus fréquemment utilises sont [2]:

<ul>
    <li>Tests de l'haleine</li>

    <i>La bactérie modifiant certaines substances dans l'estomac, il est possible de faire ingérer au patient de l'urée légèrement radioactif et de mesurer la présence des transformas.</i>
    <li>Tests sanguins.</li>

    <i>Comme les tests sanguins ne détectent uniquement la présence d'anticorps contre cette bactérie, ils deviennent inefficaces après vaccination et/ou première infections.</i>

    <li>Examen des selles.</li>

    <i>Des traces de H. Pylori peuvent être détectées dans les celles d'un patient affecte.</i>

    <li>Biopsie.</li>

    <i>La biopsie est la méthode la plus invasive mais la plus précise. Elle se fait lors d'une endoscopie. En raison de son caractère invasif, une biopsie n'est normalement utilise uniquement si le patient doit subir une endoscopie pour d'autres raisons (traitement de saignements internes, vérification de présence de cellules cancéreuses, etc.).</i>
</ul>

## Traitement
H. Pylori est traitable par des antibiotiques. Du a une prolifération de bactéries résistante aux antibiotiques, les traitements conseillés sont des traitements triple. 

Ils combinent en règle générale deux antibiotiques avec des Inhibiteur de la pompe à protons (IPP). L'action de l'IPP est de diminuer la production d'acide gastrique. 

L'IPP est indispensable à ce genre de traitement, les antibiotiques n'étant pas efficaces en milieux acides.

## Prévention
Les règles d'hygiène standard s'appliquent à la prévention contre l'infection avec H. Pylori (ce nettoyer les mains, ne pas partager les verres, ...)[4].

Dans un article publié en 2015 dans le Lancet, un groupe de médecins chinois a évalué l'efficacité d'un vaccin contre H. Pylori développé en 2005. Ils évaluent le succès de ce vaccin a environ 70% mais se montrent encore dubitatif quant au long-terme[5].
ImevaX, une start-up cofondée par l'université technique de Munich a annoncé en 2014 avoir créé un vaccin qui aurait une efficacité stérilisante de 80% dans les souris. Des tests cliniques sont prévus pour le début de l'année 2016 pour vérifier ces propriétés dans l'humain[6].

## Sources
 [1]http://www.medicinenet.com/helicobacter_pylori/article.htm
 
[2]https://www.nlm.nih.gov/medlineplus/ency/article/007501.htm 


[3]http://emedicine.medscape.com/article/176938-treatment 

[4]http://www.cdc.gov/ulcer/files/hpfacts.PDF 

[5]http://www.thelancet.com/journals/lancet/article/PIIS0140-6736%2815%2960310-5/abstract 

[6]http://www.technologist.eu/towards-the-first-helicobacter-pylori-vaccine/ 


# Réalisation
Tout d'abord, commençons par récupérer les différentes souches de la bactérie H pilory (uniquement celles correspondantes à la protéine Uréase, car on ne s'intéresse pas aux autres).

In [1]:
# Importation des librairies nécessaires
from Bio import Entrez
Entrez.email = "raphael.racine@heig-vd.ch"

In [2]:
# On s'intéresse à toutes les souches de bactérie ayant comme gène ureB (celui qui nous intéresse pour le vaccin peptidique)

query = '"Helicobacter pylori"[orgn] AND "ureB"[gene]'
handle = Entrez.esearch(db='nucleotide',retmax=1000, term=query)
record = Entrez.read(handle)
gi = record['IdList']

print gi
print len(gi)

['15644634', '987858028', '215479238', '425628494', '425626953', '815729549', '815729547', '815729545', '815729543', '815729541', '815729539', '815729537', '815729535', '815729533', '815729531', '815729529', '815729527', '815729525', '815729523', '815729521', '815729519', '815729517', '815729515', '815729513', '815729511', '815729509', '815729507', '815729505', '815729503', '815729501', '254000649', '532105813', '409896556', '409894893', '409893163', '387572303', '384561342', '384559777', '384558251', '384556702', '384551775', '380873809', '344335753', '344334211', '344331165', '325996870', '325995266', '317013424', '317011857', '317010283', '317008680', '308064373', '308062870', '308061305', '308059716', '307636682', '261838873', '261837457', '210132169', '208431905', '12057207', '297379223', '332672788', '531072757', '315585794', '443620964', '393152452', '393092936', '393031057', '393029926', '393012097', '384374920', '384374758', '384374456', '51989331', '31580720', '28372405', '19

Maintenant, nous allons créer un fichier fasta qui contiendra les séquences de chaque souche de la bactérie que nous avons trouvées précédemment.

In [3]:
from Bio import SeqIO
protein_seq = Entrez.efetch(db="nucleotide",id=gi,rettype="fasta_cds_aa",retmode="text")
protein_seq = list(SeqIO.parse(protein_seq, "fasta"))

In [4]:
# Nous devons conserver uniquement les séquences avec le gène ureB, pour ne pas avoir les autres...
updated_protein_seq = [ ps for i,ps in enumerate(protein_seq) if ps.description.find('gene=ureB')>0 ]

print updated_protein_seq[0]

ID: lcl|NC_000915.1_prot_NP_206872.1_62
Name: lcl|NC_000915.1_prot_NP_206872.1_62
Description: lcl|NC_000915.1_prot_NP_206872.1_62 [gene=ureB] [protein=urease subunit beta] [protein_id=NP_206872.1] [location=complement(75527..77236)]
Number of features: 0
Seq('MKKISRKEYVSMYGPTTGDKVRLGDTDLIAEVEHDYTIYGEELKFGGGKTLREG...SIF', SingleLetterAlphabet())


Ensuite, nous allons récupérer la séquence de base de la protéine UreB utiliséé pour l'algorithme de prédiction d'épitopes et l'aligner dans un premier temps avec les épitopes prédies.

In [2]:
input_handle = open('base_sequence_subunit_ure_b_for_epitope_prediction.fasta', 'r')

base_sequence_subunit_ureb = input_handle.readlines()[1]
print "Base sequence subunit UreB used for epitopes predictions : ", base_sequence_subunit_ureb

Base sequence subunit UreB used for epitopes predictions :  MKKISRKEYVSMYGPTTGDKVRLGDTDLIAEVEHDYTIYGEELKFGGGKTLREGMSQSNNPSKEELDLIITNALIVDYTGIYKADIGIKDGKIAGIGKGGNKDMQDGVKNNLSVGPATEALAGEGLIVTAGGIDTHIHFISPQQIPTAFASGVTTMIGGGTGPADGTNATTITPGRRNLKWMLRAAEEYSMNLGFLAKGNASNDASLADQIEAGAIGFKIHEDWGTTPSAINHALDVADKYDVQVAIHTDTLNEAGCVEDTMAAIAGRTMHTFHTEGAGGGHAPDIIKVAGEHNILPASTNPTIPFTVNTEAEHMDMLMVCHHLDKSIKEDVQFADSRIRPQTIAAEDTLHDMGIFSITSSDSQAMGRVGEVITRTWQTADKNKKEFGRLKEEKGDNDNFRIKRYLSKYTINPAIAHGISEYVGSVEVGKVADLVLWSPAFFGVKPNMIIKGGFIALSQMGDANASIPTPQPVYYREMFAHHGKAKYDANITFVSQAAYDKGIKEELGLERQVLPVKNCRNITKKDMQFNDTTAHIEVNPETYHVFVDGKEVTSKPANKVSLAQLFSIF


Création d'un fichier fasta contenant la séquence qu'on vient de lire et aussi les épitopes prédies afin de les aligner :

In [65]:
# Sauvegarder les noms d'épitopes pour les reprendre plus tard
epitopes_names = []

# Importation des librairies nécessaires pour l'alignement
from Bio.Align.Applications import MuscleCommandline
from Bio import AlignIO

# Création d'un fichier fasta contenant la séquence de base UreB et les épitopes prédies
output_handle = open('base_seq_and_epitopes.fasta', 'w')
output_handle.write(">Subunit_UreB" + '\n' + base_sequence_subunit_ureb + '\n')

# Nous devons inclure dans ce fichier FASTA les épitopes que nous avons prédies avec 
# ImmuneEpitope dans la protéine Urease afin de pouvoir les
# aligner avec les souches (fichier epitopes_immuneepitope.txt)
input_handle = open('epitopes_immuneepitope.txt', 'r')

sequences_epitopes = input_handle.readlines()

for i, seq in enumerate(sequences_epitopes):
    name = 'ImmuneEpitope_SEQ' + str(i)
    epitopes_names = epitopes_names + [name]
    output_handle.write('>' + name + '\n' + seq)

input_handle.close()

# Nous devons inclure dans ce fichier FASTA les épitopes que nous avons prédies avec SVM dans 
# la protéine Urease afin de pouvoir les
# aligner avec les souches (fichier epitopes_SMV.txt)
input_handle = open('epitopes_SVM.txt', 'r')

sequences_epitopes = input_handle.readlines()

output_handle.write('\n') # Nécessaire !!!

for i, seq in enumerate(sequences_epitopes):
    name = 'EpitopeSVM_SEQ' + str(i)
    epitopes_names = epitopes_names + [name]
    output_handle.write('>' + name + '\n' + seq)

output_handle.close()

print "FASTA containing base UreB sequences and predicted epitopes created !"

FASTA containing base UreB sequences and predicted epitopes created !


Réalisons maintenant le premier alignement :

In [66]:
# Chemin vers l'exécutable du programme muscle (à modifier si nécessaire)
# Attention ! Le   r   devant les chemins de fichier est important sur Windows !!!!!!
muscle_loc = r'muscle.exe'

# Attention aux chemins des fichiers !
in_file = r'base_seq_and_epitopes.fasta'
out_file = r'base_seq_and_epitopes_ALIGNED.aln'

souches = open('souches_bacterie_h_pilory_ureB_et_epitopes.fasta', 'r')

muscle_cline = MuscleCommandline(cmd=muscle_loc, input=in_file, out=out_file, clwstrict=True)
stdout, stderr = muscle_cline()

muscle_align = AlignIO.read(out_file, 'clustal') # this command actually performs the alignment

Ensuite, nous allons réaliser un deuxième alignement avec les souches de la bactérie

In [67]:
output_handle = open('input_for_second_alignement.fasta', 'w')

souches = open('souches_bacterie_h_pilory_ureB_et_epitopes.fasta', 'r')

for record in muscle_align:
    output_handle.write(">" + record.id + '\n' + str(record.seq) + '\n')

for line in souches.readlines():
    output_handle.write(line)

output_handle.close()
souches.close()

in_file = 'input_for_second_alignement.fasta'
out_file = 'input_for_second_alignement_ALIGNED.aln'

muscle_cline = MuscleCommandline(cmd=muscle_loc, input=in_file, out=out_file, clwstrict=True)
stdout, stderr = muscle_cline()

muscle_align = AlignIO.read(out_file, 'clustal') # this command actually performs the alignment
print muscle_align

SingleLetterAlphabet() alignment with 116 rows and 581 columns
--------------------------------------------...--- EpitopeSVM_SEQ2
--------------------------------------------...--- lcl|CP002571.1_prot_hp2017_0076_
--------------------------------------------...--- lcl|CP002572.1_prot_hp2018_0079_
--------------------------------------------...--- lcl|CP002184.1_prot_hp908_0081_8
--------------------------------------------...--- lcl|AKPT01000001.1_prot_EJC52907
--------------------------------------------...--- lcl|AKPF01000001.1_prot_EJB93677
--------------------------------------------...--- lcl|AKNX01000004.1_prot_EJB32161
--------------------------------------------...--- lcl|AKNT01000001.1_prot_EJB31289
--------------------------------------------...--- lcl|CP006691.1_prot_HPSA20_0084_
--------------------------------------------...--- lcl|CP002605.1_prot_HMPREF0462_0
--------------------------------------------...--- lcl|AVNI01000002.1_prot_EQD89528
------------------------------

Maintenant que nous pouvons tout aligner, nous pouvons calculer un score d'alignement pour chaque épitope, et choisir celles qui ont un score supérieure ou égal à une valeur alpha.

In [85]:
def compute_epitope_score(nb_lines, sequence):
    
    nb_columns = 0
    score_epitope = 0
    
    for i, acid in enumerate(sequence):       
        if acid != '-':
            score_of_column = 0 
            nb_columns = nb_columns + 1

            for aln in muscle_align:
                if aln.seq[i] != '-' and aln.seq[i] == acid:
                    score_of_column = score_of_column + 1
                    
            score_epitope = score_epitope + float(score_of_column) / nb_lines
    return score_epitope / nb_columns

# Consider good score since threshold of 0.75
threshold = 0.75

for name in epitopes_names:
    for aln in muscle_align:
        if aln.description == name:
            score = compute_epitope_score(len(muscle_align), aln.seq)
            if score >= threshold:
                print ''.join([c for c in aln.seq if c != '-']), " has a good alignement score (", score, ") (threshold =", threshold, ")"
            break; # We have to break because some sequences are in double...

MIGGGTGPADGTNATTITPGR  has a good alignement score ( 0.764778325123 ) (threshold = 0.75 )
QQIPTAFASGVTTMIGGGTG  has a good alignement score ( 0.761206896552 ) (threshold = 0.75 )
